<a href="https://colab.research.google.com/github/mingd00/aivle-miniproj6/blob/main/2%EC%B0%A8.%20%EC%9D%91%EA%B8%89%EC%83%81%ED%99%A9%20%EC%9D%B8%EC%8B%9D%20%EB%B0%8F%20%EC%9D%91%EA%B8%89%EC%8B%A4%20%EC%97%B0%EA%B3%84/3.%20%EC%9D%91%EA%B8%89%EC%8B%A4%20%EC%97%B0%EA%B3%84(%EC%B6%94%EC%B2%9C).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **응급상황 자동 인식 및 응급실 연계 서비스**
### **응급상황 연계(추천)**

* 응급실 위치와 응급전화 발신자 위치 기반 추천
* 두 좌표간 직선거리(Haversine)
    * 1) 500여 곳 응급실에 대해서, 거리 기반 가까운 응급실 찾기
    * 2) 좌표 구간을 설정하여 대상 응급실 범위를 좁힌 후, 거리 기반 가까운 응급실 찾기

## **1.환경설정**

#### 1) 구글 드라이브 연결

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
path = '/content/drive/MyDrive/project6_2/'

#### 2) 라이브러리

In [37]:
!pip install -r /content/drive/MyDrive/project6_2/requirements.txt

In [38]:
#필요한 라이브러리 설치 및 불러우기
import os
import pandas as pd
import numpy as np

from haversine import haversine
import requests
import json

### 3) 데이터 로딩

In [77]:
data = pd.read_csv(path + '응급실 정보.csv')
data.head()

,병원이름,주소,응급의료기관 종류,전화번호 1,전화번호 3,위도,경도
0,(의)내경의료재단울산제일병원,울산광역시 남구 남산로354번길 26 (신정동),응급실운영신고기관,052-220-3300,052-220-3334,35.548238,129.307011
1,(의)서일의료재단기장병원,부산광역시 기장군 기장읍 대청로72번길 6,지역응급의료기관,051-723-0171,051-723-2119,35.236029,129.216492
2,(의)성세의료재단 뉴성민병원,"인천광역시 서구 칠천왕로33번길 17 (석남동, 신석로 70(석남1동, 성민병원))",지역응급의료기관,032-726-1000,032-726-1190,37.508994,126.669479
3,(의)영문의료재단다보스병원,"경기도 용인시 처인구 백옥대로1082번길 18, 다보스종합병원 (김량장동)",지역응급의료센터,031-8021-2114,031-8021-2130,37.234641,127.210499
4,(의)효심의료재단용인서울병원,경기도 용인시 처인구 고림로 81 (고림동),지역응급의료기관,031-337-0114,031-336-0119,37.240316,127.214491


## **2. 응급실 추천**


### 1) 직선거리 계산
- haversine formula
    * Haversine은 두 지점 간의 거리를 구할 때 사용하는 수학 공식으로, 지구의 구형 구조를 고려하여 위도와 경도를 기반으로 직선 거리를 계산한다.
    * 하버사인 함수 활용: haversine((위도1, 경도1), (위도2, 경도2), unit='km')


In [78]:
display(data[:2])
haversine((data['위도'][0], data['경도'][0]), (data['위도'][1], data['경도'][1]), unit='km')

,병원이름,주소,응급의료기관 종류,전화번호 1,전화번호 3,위도,경도
0,(의)내경의료재단울산제일병원,울산광역시 남구 남산로354번길 26 (신정동),응급실운영신고기관,052-220-3300,052-220-3334,35.548238,129.307011
1,(의)서일의료재단기장병원,부산광역시 기장군 기장읍 대청로72번길 6,지역응급의료기관,051-723-0171,051-723-2119,35.236029,129.216492


35.672584868505005

### 2) 가장 가까운 응급실 3곳 추천하기1
* 세부사항
    * 입력된 좌표와 전체 응급실과 거리를 계산한 후 가장 가까운 거리의 응급실 3곳을 반환

In [79]:
def recommend_hospital(lat, lng):
  data['distance'] = data.apply(lambda x: haversine((lat, lng), (x['위도'], x['경도']), unit='km'), axis=1)
  nearest3 = data.sort_values(by='distance').head(3)
  return nearest3

recommend_hospital(37.566535, 126.977969)

,병원이름,주소,응급의료기관 종류,전화번호 1,전화번호 3,위도,경도,distance
23,강북삼성병원,서울특별시 종로구 새문안로 29 (평동),지역응급의료센터,02-2001-2001,02-2001-1000,37.568498,126.967938,0.910636
180,서울적십자병원,"서울특별시 종로구 새문안로 9, 적십자병원 (평동)",지역응급의료기관,02-2002-8000,02-2002-8888,37.567155,126.966999,0.969365
195,세란병원,서울특별시 종로구 통일로 256 (무악동),지역응급의료기관,02-737-0181,02-737-0181,37.575340,126.957707,2.036535


### 3) 가장 가까운 응급실 3곳 추천하기2
* 문제점 : 입력 받은 좌표와 응급실 전체와의 거리를 모두 계산하는 것은 비효율적
* 해결 방안 : 입력 받은 좌표를 기준으로 일정 범위 내에 해당되는 응급실에 대해서 거리를 계산하고 추천하도록 기존 함수를 수정
    * 입력 받은 위도, 경도 값에 ± α 하여 일정 범위 구간을 정하고, 응급실 정보에서 해당 구간을 먼저 조회한 후 거리 계산

In [86]:
def recommend_hospital2(lat, lng, range):
  filtered_data = data[data['위도'].between(lat - range, lat + range) & data['경도'].between(lng - range, lng + range)].copy()
  print(len(filtered_data))
  filtered_data['distance'] = filtered_data.apply(lambda x: haversine((lat, lng), (x['위도'], x['경도']), unit='km'), axis=1)
  nearest3 = filtered_data.sort_values(by='distance').head(3)
  return nearest3

recommend_hospital2(37.568498, 126.967938, 0.01)

2


,병원이름,주소,응급의료기관 종류,전화번호 1,전화번호 3,위도,경도,distance
23,강북삼성병원,서울특별시 종로구 새문안로 29 (평동),지역응급의료센터,02-2001-2001,02-2001-1000,37.568498,126.967938,0.000041
180,서울적십자병원,"서울특별시 종로구 새문안로 9, 적십자병원 (평동)",지역응급의료기관,02-2002-8000,02-2002-8888,37.567155,126.966999,0.170716


### 4) naver 지도 api 활용하여 거리 계산
* 두 지점간, 최단 도로거리, 소요 시간을 계산하는 함수
    * 입력 : 두 지점의 위도, 경도, 네이버클라우드id, 암호키
    * 출력 : 도로거리(km)

* 네이버 Maps API 활용
    * 사용할 API : Direction 5
    * 가이드 : https://guide.ncloud-docs.com/docs/ko/maps-direction5-api
    * params의 옵션: 'trafast' (실시간 빠른 길 옵션)

- maps 클라이언트ID, 키 로딩

In [87]:
c_id = 'citqgne3pt'
c_key = 'a2kcwhrtLmEe4jyufRv69W8P3hFsIIPBopi8czZM'

-  함수 생성

In [88]:
def get_dist(start_lat, start_lng, dest_lat, dest_lng, c_id, c_key):
    url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": c_id,
        "X-NCP-APIGW-API-KEY": c_key,
    }
    params = {
        "start": f"{start_lng},{start_lat}",  # 출발지 (경도, 위도)
        "goal": f"{dest_lng},{dest_lat}",    # 목적지 (경도, 위도)
        "option": "trafast"  # 실시간 빠른 길 옵션
    }

    # 요청하고, 답변 받아오기
    response = requests.get(url, params, headers=headers)
    response = response.json()

    if response['code'] == 1:
      dist = 0
    else:
      dist = response['route']['trafast'][0]['summary']['distance'] # m(미터)

    return dist

* 테스트

In [89]:
get_dist(37.566535, 126.977969, 37.566535, 126.977969, c_id, c_key)

0

* recommend_hospital2 함수를 참조해서 recommend_hospital3 만들기
    * 거리 계산 부분을 get_dist 함수로 대체
    * 입력 부분 수정

In [92]:
def recommend_hospital3(data, lat, lng, range, c_id, c_key) :
  li = []

  # 특정 범위 데이터만 추출
  filtered_data = data[data['위도'].between(lat - range, lat + range) & data['경도'].between(lng - range, lng + range)].copy()

  # 각 병원까지의 거리를 계산하여 컬럼 생성
  for d in filtered_data.iterrows():
    li.append([d[1]['병원이름'], get_dist(lat, lng, d[1]['위도'], d[1]['경도'], c_id, c_key)])

  # 상위 3개 추출
  li = sorted(li, key=lambda x: (x[1]))[:3]
  print('가장 가까운 병원 TOP3')
  for l in li:
    print(f'이름: {l[0]}, 거리: {l[1]}m')

In [95]:
recommend_hospital3(data, 34.8092145829474, 126.4163391650001, 0.02, c_id, c_key)

가장 가까운 병원 TOP3
이름: 목포한국병원, 거리: 0m
이름: 목포기독병원, 거리: 1360m
이름: 의료법인목포구암의료재단 목포중앙병원, 거리: 1577m
